In [6]:
import geemap
import ee
from scripts.analysis_functions import bs_calc_new
ee.Initialize(project='pyregence-ee')

In [7]:
fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_2020_20230406")
fire = fires.first() # ee.Feature(fires.filter(ee.Filter.eq('Name','Dixie'))
print(fire.getInfo()['properties'])

bs = bs_calc_new(fire)

{'Asmnt_Type': 'Initial', 'BurnBndAc': 1164, 'BurnBndLat': '35.574', 'BurnBndLon': '-96.168', 'Discovery': '2020-01-01', 'Event_ID': 'OK3555209617620200101', 'Incid_Name': 'BRIAR CREEK', 'Incid_Type': 'Wildfire', 'Map_ID': 10018965, 'Map_Prog': 'MTBS', 'Year': 2020, 'irwinID': 'AFAA0704-7DA5-4CFE-AB74-9DA456AC0B50'}


In [9]:
Map = geemap.Map()
Map.addLayer(bs,{'min':1,'max':4,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'bs one fire')
Map.centerObject(bs,10)

Map



Map(center=[35.57497943507915, -96.16976311652586], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
## Deconstruct the functions that go into bs_calc_new() if you need to dig down deeper 

import scripts.fire_info as fi
import scripts.get_image_collections as gic
import scripts.get_image_collections2 as gic2
import scripts.analysis_functions as af
def bs_calc_new_deconstructed(feat: ee.Feature):
    fire = ee.Feature(feat)
    # fire_geom = fire.geometry()
    
    fire = ee.Feature(fi.set_windows(fire))
    region = fire.geometry()
    # mode = fire.get('mode')
    pre_start = fire.get('pre_start')
    pre_end = fire.get('pre_end')
    post_start = fire.get('post_start')
    post_end = fire.get('post_end')
    
    sensor = "landsat"
    pre_collection = gic2.getLandsatToa(pre_start,pre_end,region)
    pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)

    post_collection = gic2.getLandsatToa(post_start,post_end,region)
    post_img = gic.get_composite(post_collection,gic.make_nrt_composite, sensor) 
    
    rdnbr_calc = af.rdnbr(pre_img,post_img)
    miller = af.miller_thresholds(rdnbr_calc)
    
    return ee.Image(miller).clip(region).select('MillersThresholds').toByte() 
    # return pre_start, pre_end, post_start, post_end, pre_collection, post_collection, pre_img, post_img, 

pre_s,pre_e,post_s,post_e,pre_c,post_c,pre_i,post_i = bs_calc_new_deconstructed(fire)

In [ ]:
# DEBUG NIFC single fire
# One set of yearly fires, no need for remove_recent() or year/ac filtering, that was already done upon data fetch
## BUG: when mode is set to recent, get error on:  post_collection = gic2.getLandsatToa(post_start,post_end,region)
# yr='2021'
# # fires = ee.FeatureCollection("projects/pyregence-ee/assets/conus/mtbs/mtbs_fires_all_20220928")
# fires = ee.FeatureCollection(f"projects/pyregence-ee/assets/conus/nifc/nifc_fires_{yr}_gte500acres_20221004")

# print(f'Total Fires in FeatureCollection: {fires.size().getInfo()}')

# # filter 2022 fires for a July 1 cut-off
# if yr=='2022':
#     fires = fires.filter(ee.Filter.lte('Discovery','2022-07-01'))
#     print(fires.sort('Discovery',False).first().getInfo()['properties'])
#     print(f'Fires after filter: {fires.size().getInfo()}')

# # fire= fires.filter(ee.Filter.eq('GlobalID','826b1cca-9f48-4078-9442-c9bc715b9e44')).first()
# fire = fires.sort('Discovery',False).first()
# # print(fire.getInfo()['properties'])
# # finding that cut-off b/w a fire that would be assigned recent vs historical (365 days prior to today)
# # fire = ee.Feature(fires.filter(ee.Filter.lte('Discovery','2021-11-01')).sort('Discovery',False).first())

# # we now get run mode and pre/post dates set for the fire feature itself
# fire = ee.Feature(fi.set_windows(fire))
## region = fire.geometry()
# mode = fire.get('mode')
# pre_start = fire.get('pre_start')
# pre_end = fire.get('pre_end')
# post_start = fire.get('post_start')
# post_end = fire.get('post_end')
# print(mode.getInfo())
# print(fire.getInfo()['properties'])

# sensor = "landsat"
# pre_collection = gic2.getLandsatToa(pre_start,pre_end,region)
# pre_img = gic.get_composite(pre_collection,gic.make_pre_composite,pre_start,pre_end)

# post_collection = gic2.getLandsatToa(post_start,post_end,region)
# post_img = gic.get_composite(post_collection,gic.make_nrt_composite, sensor) 

# print(pre_collection.size().getInfo())
# print(pre_img.bandNames().getInfo())

# print(post_collection.size().getInfo())
# print(post_img.bandNames().getInfo())

# rdnbr = af.rdnbr(pre_img,post_img)
# miller = af.miller_thresholds(rdnbr).add(1).clip(region).select('MillersThresholds').toByte()     

In [ ]:
# look at post_img and determine for yourself if its good enough to be using for analysis 
# Map.addLayer(pre_img,{"bands":["SWIR2","NIR","RED"],"min":0,"max":0.848},"pre")
# Map.addLayer(post_img,{"bands":["SWIR2","NIR","RED"],"min":0,"max":0.848},"post")
# Map.addLayer(rdnbr.clip(region),{"min":0,"max":1500,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'rdnbr')
# Map.addLayer(miller.clip(region),{"min":1,"max":4,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'miller')
# Map.addLayer(region, {}, 'region')
# Map.centerObject(region, 11)
# Map

In [ ]:
# Map = geemap.Map()
# Map.addLayer(pre_img.clip(region),{"bands":["SWIR2","NIR","RED"],"min":0,"max":0.848},"pre snr")
# Map.addLayer(post_img.clip(region),{"bands":["SWIR2","NIR","RED"],"min":0,"max":0.848},"post snr")
# Map.addLayer(pre_img.clip(region),{"bands":["RED","BLUE","GREEN"],"min":0,"max":0.848},"pre rgb")
# Map.addLayer(post_img.clip(region),{"bands":["RED","BLUE","GREEN"],"min":0,"max":0.848},"post rgb")
# Map.addLayer(rdnbr.clip(region),{"min":0,"max":1500,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'rdnbr')
# Map.addLayer(miller.clip(region),{"min":1,"max":4,"palette":["00ff1f","fbff0e","ffbc00","ff0000"]},'miller')
# Map.addLayer(region, {}, 'region')
# Map.centerObject(region, 11)
# Map